In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('emails.csv')

In [3]:
df.head(1)

,text,spam,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109
0,Subject: naturally irresistible your corporate...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(5731, 110)

In [5]:
df=df.iloc[:,0:2]

In [6]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [7]:
df.isnull().sum()

text    0
spam    2
dtype: int64

In [8]:
df.dropna(subset=['spam'],inplace=True)

In [9]:
df.duplicated().sum()

33

In [10]:
df=df.drop_duplicates()

In [11]:
df.shape

(5696, 2)

In [12]:
df.iloc[15]['text']

'Subject: search engine position  be the very first listing in the top search engines immediately .  our company will now place any business with a qualified website  permanently at the top of the major search engines guaranteed never to move  ( ex : yahoo ! , msn , alta vista , etc . ) . this promotion includes unlimited  traffic and is not going to last long . if you are interested in being  guaranteed first position in the top search engines at a promotional fee ,  please contact us promptly to find out if you qualify via email at  searchl 1 @ telefonica . net . pe it \' s very important to include the url ( s ) if you  are interested in promoting ! ! ! this is not pay per click . examples will  be provided .  this promotion is only valid in the usa and canada .  sincerely ,  the search engine placement specialists  if you wish to be removed from this list , please respond to the following  email address and type the word " remove " in your subject line :  search 6 @ speedy . com . 

In [13]:
#Text preprocessing

#Step-1.Convert to small letters(i.e. lowercase)
#Step-2.remove special chars
#Step-3.remove stopwords(i.e. a,of,the)
#Step-4.convert to vectors

In [14]:
#Step-1
df['text']=df['text'].apply(lambda x:x.lower())

In [15]:
df.head()

,text,spam
0,subject: naturally irresistible your corporate...,1
1,subject: the stock trading gunslinger fanny i...,1
2,subject: unbelievable new homes made easy im ...,1
3,subject: 4 color printing special request add...,1
4,"subject: do not have money , get software cds ...",1


In [16]:
#Step-2
def remove_special_chars(text):
    result=""
    for i in text:# i is each character in text
        if i.isalnum() or i==' ':
            result+=i
    return result.strip()
        

In [17]:
df['text']=df['text'].apply(remove_special_chars)

In [18]:
df['spam'].value_counts()

0                                                                                               4326
1                                                                                               1368
 its termination would not  have such a phenomenal impact on the power situation .  however        1
 mr suresh prabhu                                                                                  1
Name: spam, dtype: int64

In [19]:
df[df['spam'].str.contains('its termination would not  have such a phenomenal impact on the power situation .  however')]

,text,spam
1381,e dpc contributed only 0 7 per cent of the t...,its termination would not have such a phenom...


In [20]:
df[df['spam'].str.contains('mr suresh prabhu')]

,text,spam
2652,lf against undeserved claims in the event of e...,mr suresh prabhu


In [21]:
# so in order to convert the spam column into int, we need to drop two rows 1381 and 2652
df.drop([1381,2652],axis=0,inplace=True)#axis=0 means rows

In [22]:
#Step-3 & 4
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
cv=CountVectorizer(stop_words='english',max_features=10000) # stop_words='english' we need to specify the language whose stop_words we want to remove and max_features=10000 because we want only 10000 unique words i.e. 10000 columns and cv is vector object

In [24]:
cv.fit_transform(df['text']).toarray().shape # since cv.fit_transform(df['text']) returns sparse matrix so we are converting it to numpy array and thus we are transforming each row(i.e. email) into vectors

(5694, 10000)

In [25]:
# now there are 5694 vectors and 10000 columns of unique words

In [26]:
X=cv.fit_transform(df['text']).toarray()
X.shape

(5694, 10000)

In [27]:
X # finally all the text has now become vector

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
X[0] # it is the first email vector and X[0] is an array which has 10000 columns 

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
df['spam']=df['spam'].astype('int')

In [30]:
y=df['spam'].values
y

array([1, 1, 1, ..., 0, 0, 0])

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [32]:
from sklearn.naive_bayes import MultinomialNB

In [33]:
clf=MultinomialNB()

In [34]:
clf.fit(X_train,y_train)

MultinomialNB()

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
y_pred=clf.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 0, 0])

In [37]:
accuracy_score(y_test,y_pred)

0.9938542581211589

In [38]:
import pickle

In [39]:
pickle.dump(cv,open('cv.pkl','wb'))

In [40]:
pickle.dump(clf,open('clf.pkl','wb'))